In [1]:
!pip install contractions
!gdown '1N7CLrqNuOPuGoisB41N7kA645U2y2HAd'
!gunzip /content/amazon_reviews_us_Watches_v1_00.tsv.gz

Downloading...
From (original): https://drive.google.com/uc?id=1N7CLrqNuOPuGoisB41N7kA645U2y2HAd
From (redirected): https://drive.google.com/uc?id=1N7CLrqNuOPuGoisB41N7kA645U2y2HAd&confirm=t&uuid=8daaa005-75d8-4ef7-a7d6-8f4c1a6d280d
To: /content/amazon_reviews_us_Watches_v1_00.tsv.gz
100% 163M/163M [00:01<00:00, 125MB/s]
gzip: /content/amazon_reviews_us_Watches_v1_00.tsv already exists; do you wish to overwrite (y or n)? ^C


In [2]:
import pandas as pd
import torch
import scipy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from torch import nn, optim

from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, ConfusionMatrixDisplay


from torch import nn, optim
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
import pandas as pd
df = pd.read_csv('/content/amazon_reviews_us_Watches_v1_00.tsv', delimiter='\t', on_bad_lines='skip')
X, y = df['review_body'], df['star_rating']

In [4]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


## **Data Cleaning**

In [5]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import contractions

nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return ' '.join(filtered_tokens)

def normalize(text):
    return str(text).lower()

def remove_tags(text): # some tags like <br> ...
    text = re.sub('<.*?>', '', text)
    text = re.sub('\[.*?\]', '', text)
    return text

def remove_emojies(text):
    text = text.encode('ascii', 'ignore').decode('ascii')
    return text

def remove_punctuations(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def remove_spaces(text):
    text = ' '.join(text.split())
    return text

def remove_numbers(text):
    text = re.sub(r'\d+', '', text)
    return text

def fix_contractions(text): # can't -> cannot
    return contractions.fix(text)

def remove_one_char(text):
    words = text.split()
    cleaned_words = [word for word in words if len(word) > 2]
    text = ' '.join(cleaned_words)
    return text

def clean_text(text):
    text = normalize(text)
    text = fix_contractions(text)
    text = remove_stopwords(text)
    text = remove_tags(text)
    text = remove_emojies(text)
    text = remove_punctuations(text)
    text = remove_numbers(text)
    text = remove_spaces(text)
    text = remove_one_char(text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **Bert Transfer Learning**

In [6]:
!pip install datasets transformers evaluate accelerate

In [7]:
!gdown 1AERCGs8VaK321NLglvheiKGmA4A55lLS

Downloading...
From (original): https://drive.google.com/uc?id=1AERCGs8VaK321NLglvheiKGmA4A55lLS
From (redirected): https://drive.google.com/uc?id=1AERCGs8VaK321NLglvheiKGmA4A55lLS&confirm=t&uuid=ed8d3c0b-3a73-43e0-a350-efeb8ffa6df8
To: /content/data.csv
100% 160M/160M [00:04<00:00, 38.0MB/s]


In [12]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/data.csv").drop("Unnamed: 0", axis=1).reset_index(drop=True)
df = df.dropna()
X, y = df['review_body'], df['star_rating']
y = y.apply(lambda x: 0 if x <= 2 else 1 if x == 3 else 2)

label2id = {"Negative": 0, "Neutral": 1, "Postiive": 2} #Need for bert models
id2label = {0: "Negative", 1: "Neutral", 2: "Postiive"}

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.6, shuffle=True, random_state=42, stratify=y)
X_test, X_validation, y_test, y_validation = train_test_split(X_validation, y_validation, train_size=0.5, shuffle=True, random_state=42, stratify=y_validation)

In [13]:
from datasets import Dataset, DatasetDict

ds_train = Dataset.from_pandas(pd.concat([X_train, y_train], axis=1, keys=['text', 'label']).reset_index(drop=True))
ds_train = ds_train.train_test_split(test_size=0.1)['test']
ds_validation = Dataset.from_pandas(pd.concat([X_validation, y_validation], axis=1, keys=['text', 'label']).reset_index(drop=True))
ds_test = Dataset.from_pandas(pd.concat([X_test, y_test], axis=1, keys=['text', 'label']).reset_index(drop=True))
ds = DatasetDict({'train': ds_train, 'validation': ds_validation, 'test': ds_test})

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(id2label), id2label=id2label, label2id=label2id
)
for name, param in model.named_parameters():
    if name.startswith('roberta'):
        param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import numpy as np

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

def compute_metrics(eval_pred): # For cal accuracy
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

ds = ds.map(preprocess_function, batched=True)
accuracy = evaluate.load("accuracy")

Map:   0%|          | 0/57374 [00:00<?, ? examples/s]

Map:   0%|          | 0/191244 [00:00<?, ? examples/s]

Map:   0%|          | 0/191244 [00:00<?, ? examples/s]

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(ds['test'])

## **Neural Network**

In [ ]:
!pip install datasets

In [21]:
!gdown 1AERCGs8VaK321NLglvheiKGmA4A55lLS

Downloading...
From (original): https://drive.google.com/uc?id=1AERCGs8VaK321NLglvheiKGmA4A55lLS
From (redirected): https://drive.google.com/uc?id=1AERCGs8VaK321NLglvheiKGmA4A55lLS&confirm=t&uuid=2da4469e-fde0-46ec-a0d3-67deebc21d2d
To: /content/data.csv
100% 160M/160M [00:01<00:00, 99.1MB/s]


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("data.csv").drop("Unnamed: 0", axis=1).reset_index(drop=True)
df = df.dropna()
X, y = df['review_body'], df['star_rating']
y = y.apply(lambda x: 0 if x <= 2 else 1 if x == 3 else 2)
label2id = {"Negative": 0, "Neutral": 1, "Postiive": 2}
id2label = {0: "Negative", 1: "Neutral", 2: "Postiive"}
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.6, shuffle=True, random_state=42, stratify=y)
X_test, X_validation, y_test, y_validation = train_test_split(X_validation, y_validation, train_size=0.5, shuffle=True, random_state=42, stratify=y_validation)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X_train_transformed = vectorizer.fit_transform(X_train, y_train)
X_validation_transformed = vectorizer.transform(X_validation)
X_test_transformed = vectorizer.transform(X_test)

In [24]:
import torch
import scipy

X_train_transformed = torch.tensor(scipy.sparse.csr_matrix.todense(X_train_transformed)).float()
X_test_transformed = torch.tensor(scipy.sparse.csr_matrix.todense(X_test_transformed)).float()

y_train_transformed = torch.tensor(y_train.values)
y_test_transformed = torch.tensor(y_test.values)

In [25]:
from torch import nn, optim

model = nn.Sequential(
    nn.Linear(1000, 32),
    nn.ReLU(),
    nn.Linear(32, 3),
    nn.LogSoftmax(dim=1)
).cuda()

criterion = nn.NLLLoss().cuda()
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.1)

In [26]:
from sklearn.metrics import precision_recall_fscore_support
epochs = 500
for e in range(epochs):
    optimizer.zero_grad()
    output = model.forward(X_train_transformed.cuda())
    loss = criterion(output, y_train_transformed.cuda())
    loss.backward()
    optimizer.step()
    prec, rec, f1, sup = precision_recall_fscore_support(y_train_transformed.cpu().detach().numpy(), output.argmax(axis=1).cpu().detach().numpy())
    accuracy = accuracy_score(y_train_transformed.cpu().detach().numpy(), output.argmax(axis=1).cpu().detach().numpy())
    print(f"train loss: {loss.item():.2}, train_accuracy: {accuracy * 100:.3}, precision: {prec[0]:.2}, recall: {rec[0]:.2}, f1: {f1[0]:.2}")
    if e % 10 == 0:
        with torch.no_grad():
            model.eval()
            log_ps = model(X_test_transformed.cuda())
            test_loss = criterion(log_ps, y_test_transformed.cuda())
            prec, rec, f1, sup = precision_recall_fscore_support(y_test_transformed.cpu().detach().numpy(), log_ps.argmax(axis=1).cpu().detach().numpy())
            accuracy = accuracy_score(y_test_transformed.cpu().detach().numpy(), log_ps.argmax(axis=1).cpu().detach().numpy())
            print(f"test loss: {loss.item():.2}, test_accuracy: {accuracy * 100:.3}, precision: {prec[0]:.2}, recall: {rec[0]:.2}, f1: {f1[0]:.2}")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train loss: 1.1, train_accuracy: 74.7, precision: 0.0, recall: 0.0, f1: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


test loss: 1.1, test_accuracy: 77.5, precision: 0.0, recall: 0.0, f1: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

train loss: 0.43, train_accuracy: 84.4, precision: 0.71, recall: 0.6, f1: 0.65


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train loss: 0.43, train_accuracy: 84.5, precision: 0.7, recall: 0.62, f1: 0.66


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train loss: 0.42, train_accuracy: 84.7, precision: 0.68, recall: 0.65, f1: 0.66
train loss: 0.42, train_accuracy: 84.7, precision: 0.67, recall: 0.66, f1: 0.67
train loss: 0.42, train_accuracy: 84.8, precision: 0.67, recall: 0.68, f1: 0.67
train loss: 0.42, train_accuracy: 84.9, precision: 0.66, recall: 0.68, f1: 0.67
train loss: 0.41, train_accuracy: 84.9, precision: 0.67, recall: 0.69, f1: 0.68
train loss: 0.41, train_accuracy: 85.0, precision: 0.67, recall: 0.69, f1: 0.68
test loss: 0.41, test_accuracy: 85.0, precision: 0.68, recall: 0.68, f1: 0.68
train loss: 0.41, train_accuracy: 85.0, precision: 0.68, recall: 0.68, f1: 0.68
train loss: 0.41, train_accuracy: 85.1, precision: 0.68, recall: 0.68, f1: 0.68
train loss: 0.41, train_accuracy: 85.1, precision: 0.68, recall: 0.68, f1: 0.68
train loss: 0.4, train_accuracy: 85.1, precision: 0.69, recall: 0.68, f1: 0.68
train loss: 0.4, train_accuracy: 85.2, precision: 0.69, recall: 0.68, f1: 0.68
train loss: 0.4, train_accuracy: 85.2, preci

In [27]:
#***Logistic Regression***

In [28]:
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, ConfusionMatrixDisplay

In [29]:
df = pd.read_csv("data.csv").drop("Unnamed: 0", axis=1).reset_index(drop=True)
df = df.dropna()
X, y = df['review_body'], df['star_rating']
y = y.apply(lambda x: 0 if x <= 2 else 1 if x == 3 else 2)
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.6, shuffle=True, random_state=42, stratify=y)
X_test, X_validation, y_test, y_validation = train_test_split(X_validation, y_validation, train_size=0.5, shuffle=True, random_state=42, stratify=y_validation)

In [30]:
vectorizer = TfidfVectorizer()
X_train_1 = [str(text) for text in X_train] # str first
X_train_1 = vectorizer.fit_transform(X_train_1)
model = LogisticRegression()
model.fit(X_train_1, y_train)
X_validation_1 = vectorizer.transform(X_validation)
y_pred = model.predict(X_validation_1)
accuracy_t = accuracy_score(y_validation, y_pred)
precision, recall, fscore, support = precision_recall_fscore_support(y_validation, y_pred)
print("Accuracy Test:", accuracy_t)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", fscore)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy Test: 0.8670912551504883
Precision: [0.73351538 0.42351901 0.90272833]
Recall: [0.73595382 0.12206932 0.97023769]
F1-score: [0.73473258 0.18951533 0.93526635]


## **Test**

In [ ]:
for i in X_train:

In [ ]:
import numpy as np
np.sort(a.values)

In [ ]:
X_train_transformed

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Load and preprocess the data
df = pd.read_csv("data.csv").drop("Unnamed: 0", axis=1).reset_index(drop=True)
df = df.dropna()
X, y = df['review_body'], df['star_rating']
y = y.apply(lambda x: 0 if x <= 2 else 1 if x == 3 else 2)
label2id = {"Negative": 0, "Neutral": 1, "Positive": 2}
id2label = {0: "Negative", 1: "Neutral", 2: "Positive"}
X_train, X_validation, y_train, y_validation = train_test_split(
    X, y, train_size=0.6, shuffle=True, random_state=42, stratify=y
)
X_test, X_validation, y_test, y_validation = train_test_split(
    X_validation, y_validation, train_size=0.5, shuffle=True, random_state=42, stratify=y_validation
)

In [ ]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_validation_sequences = tokenizer.texts_to_sequences(X_validation)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
max_length = 100  # Define the desired maximum length of sequences

X_train_padded = pad_sequences(X_train_sequences, padding='post', maxlen=max_length)
X_validation_padded = pad_sequences(X_validation_sequences, padding='post', maxlen=max_length)
X_test_padded = pad_sequences(X_test_sequences, padding='post', maxlen=max_length)

vocab_size = len(tokenizer.word_index) + 1

# Convert data to tensors
X_train_padded = torch.tensor(X_train_padded).long()
X_validation_padded = torch.tensor(X_validation_padded).long()
X_test_padded = torch.tensor(X_test_padded).long()

y_train_transformed = torch.tensor(y_train.values)
y_test_transformed = torch.tensor(y_test.values)

In [ ]:
# Define the model architecture
embedding_dim = 32

model = nn.Sequential(
    nn.Embedding(vocab_size, embedding_dim),
    nn.Flatten(),
    nn.Linear(max_length * embedding_dim, 32),
    nn.ReLU(),
    nn.Linear(32, 3),
    nn.LogSoftmax(dim=1)
).cuda()

criterion = nn.NLLLoss().cuda()
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.1)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

epochs = 500

for e in range(epochs):
    optimizer.zero_grad()
    output = model.forward(X_train_padded.cuda())
    loss = criterion(output, y_train_transformed.cuda())
    loss.backward()
    optimizer.step()
    prec, rec, f1, sup = precision_recall_fscore_support(
        y_train_transformed.cpu().detach().numpy(), output.argmax(axis=1).cpu().detach().numpy()
    )
    print(f"train loss: {loss.item():.2}, precision: {prec[0]:.2}, recall: {rec[0]:.2}, f1: {f1[0]:.2}")
    if e % 10 == 0:
        with torch.no_grad():
            model.eval()
            log_ps = model(X_test_padded.cuda())
            test_loss = criterion(log_ps, y_test_transformed.cuda())
            prec, rec, f1, sup = precision_recall_fscore_support(
                y_test_transformed.cpu().detach().numpy(), log_ps.argmax(axis=1).cpu().detach().numpy()
            )
            print(f"test loss: {loss.item():.2}, precision: {prec[0]:.2}, recall: {rec[0]:.2}, f1: {f1[0]:.2}")

## ***Embedding 2***

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("data.csv").drop("Unnamed: 0", axis=1).reset_index(drop=True)
df = df.dropna()
X, y = df['review_body'], df['star_rating']
y = y.apply(lambda x: 0 if x <= 2 else 1 if x == 3 else 2)
label2id = {"Negative": 0, "Neutral": 1, "Postiive": 2}
id2label = {0: "Negative", 1: "Neutral", 2: "Postiive"}
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.6, shuffle=True, random_state=42, stratify=y)
X_test, X_validation, y_test, y_validation = train_test_split(X_validation, y_validation, train_size=0.5, shuffle=True, random_state=42, stratify=y_validation)

In [ ]:
from datasets import Dataset, DatasetDict

ds_train = Dataset.from_pandas(pd.concat([X_train, y_train], axis=1, keys=['text', 'label']).reset_index(drop=True))
ds_train = ds_train.train_test_split(test_size=0.1)['test']
ds_validation = Dataset.from_pandas(pd.concat([X_validation, y_validation], axis=1, keys=['text', 'label']).reset_index(drop=True))
ds_test = Dataset.from_pandas(pd.concat([X_test, y_test], axis=1, keys=['text', 'label']).reset_index(drop=True))
ds = DatasetDict({'train': ds_train, 'validation': ds_validation, 'test': ds_test})

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertModel.from_pretrained(
    'bert-base-uncased', num_labels=len(id2label), id2label=id2label, label2id=label2id
)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128, return_tensors="pt")
ds = ds.map(preprocess_function, batched=True)
ds.set_format("pt")

In [ ]:
from torch import nn, optim

m = nn.Sequential(
    model.embeddings,
    nn.Flatten(),
    nn.Linear(98304, 32),
    nn.ReLU(),
    nn.Linear(32, 3),
    nn.LogSoftmax(dim=1)
).cuda()
for name, param in m.named_parameters():
    if not name.startswith('0'):
        param.requires_grad = False

criterion = nn.NLLLoss().cuda()
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.1)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 32
train_loader = DataLoader(ds['train'], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(ds['test'], batch_size=batch_size)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
import torch

epochs = 500
for e in range(epochs):
    train_loss = 0
    for i, data in enumerate(train_loader):
        inputs, targets = data['input_ids'].cuda(), data['label'].cuda()
        optimizer.zero_grad()
        output = m.forward(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    print(f"Epoch {e+1}/{epochs}, train_loss: {train_loss:.2}")

    if e % 10 == 0:
        with torch.no_grad():
            m.eval()
            test_loss = 0
            ans = []
            for data in test_loader:
                inputs, targets = data['input_ids'].cuda(), data['label'].cuda()
                log_ps = m(inputs)
                test_loss += criterion(log_ps, targets).item()
                ans.append(log_ps.argmax(axis=1).cpu().detach().numpy())
            test_loss /= len(test_loader)

            ans = np.concatenate(ans, axis=0)
            prec, rec, f1, sup = precision_recall_fscore_support(y_test.values, ans)
            accuracy = accuracy_score(y_test.values, ans)
            print(f"Epoch {e+1}/{epochs}, train_loss: {train_loss:.2}, test_loss: {test_loss:.2}, test_accuracy: {accuracy * 100:.3}, precision: {prec[0]:.2}, recall: {rec[0]:.2}, f1: {f1[0]:.2}")